<a href="https://colab.research.google.com/github/cabamarcos/P1-RRNN/blob/main/P1_RRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 1 Redes de Neuronas

## 1. Introducción

## 2. Programación del modelo lineal Adaline

In [56]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split


Cargamos los datos en un mismo dataframe para posteriormente poder aleatorizarlos

In [57]:
archivos_csv = ['gt_2011.csv', 'gt_2012.csv', 'gt_2013.csv', 'gt_2014.csv', 'gt_2015.csv']

df_final = pd.DataFrame()

for archivo in archivos_csv:
    # Leer el archivo CSV y cargarlo en un DataFrame temporal
    df_temporal = pd.read_csv(archivo)

    # Concatenar el DataFrame temporal con el DataFrame final
    df_final = pd.concat([df_final, df_temporal], ignore_index=True)


Ahora aleatorizamos los datos

In [58]:
datos = df_final.sample(frac=1, random_state=random.seed(123))
datos

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
14263,8.1883,1016.7,93.811,2.7254,23.790,1084.1,550.09,134.21,11.794,2.04630,68.764
16321,14.1490,1017.6,83.055,3.0875,24.218,1086.0,549.99,133.79,11.872,2.55650,76.899
31759,21.0200,1019.0,56.600,4.0826,33.526,1100.0,541.02,151.21,13.500,3.09440,66.549
2347,13.2010,1009.5,100.120,3.5562,22.370,1074.4,549.84,125.52,11.351,1.70050,67.606
16316,14.9490,1018.7,87.213,4.1582,32.522,1100.0,530.30,157.34,13.877,0.99215,67.841
...,...,...,...,...,...,...,...,...,...,...,...
7944,3.9947,1019.0,94.028,5.9859,34.269,1100.0,524.97,167.40,14.303,0.37583,61.567
12044,21.1140,1016.6,83.783,3.4022,20.008,1058.4,549.89,110.81,10.552,4.06080,54.041
13719,21.6530,1007.9,83.814,3.6172,19.798,1056.6,550.11,109.45,10.496,5.16550,55.170
28595,7.0983,1020.3,83.139,2.5173,21.441,1063.3,549.97,122.64,11.172,2.52260,72.596


In [59]:
# Verificar si hay valores nulos
if datos.isnull().values.any():
    print("Hay valores nulos en la matriz.")
else:
    print("No hay valores nulos en la matriz.")

No hay valores nulos en la matriz.


In [60]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36733 entries, 14263 to 16471
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      36733 non-null  float64
 1   AP      36733 non-null  float64
 2   AH      36733 non-null  float64
 3   AFDP    36733 non-null  float64
 4   GTEP    36733 non-null  float64
 5   TIT     36733 non-null  float64
 6   TAT     36733 non-null  float64
 7   TEY     36733 non-null  float64
 8   CDP     36733 non-null  float64
 9   CO      36733 non-null  float64
 10  NOX     36733 non-null  float64
dtypes: float64(11)
memory usage: 3.4 MB


In [61]:
for columna in datos:
  min = datos[columna].min()
  max = datos[columna].max()
  datos[columna] = (datos[columna] - min)/(max - min)

print(datos)

             AT        AP        AH      AFDP      GTEP       TIT       TAT  \
14263  0.332806  0.607882  0.916061  0.115513  0.264662  0.832168  0.986859   
16321  0.470347  0.625616  0.774749  0.181073  0.283257  0.851149  0.984332   
31759  0.628892  0.653202  0.427183  0.361240  0.687636  0.991009  0.757645   
2347   0.448472  0.466010  0.998949  0.265933  0.202972  0.735265  0.980541   
16316  0.488807  0.647291  0.829377  0.374928  0.644018  0.991009  0.486732   
...         ...       ...       ...       ...       ...       ...       ...   
7944   0.236041  0.653202  0.918912  0.705841  0.719915  0.991009  0.352034   
12044  0.631061  0.605911  0.784313  0.238050  0.100356  0.575425  0.981804   
13719  0.643498  0.434483  0.784720  0.276977  0.091233  0.557443  0.987364   
28595  0.307655  0.678818  0.775852  0.077835  0.162612  0.624376  0.983826   
16471  0.389378  0.623645  0.672390  0.383274  0.671996  0.991009  0.423048   

            TEY       CDP        CO       NOX  
142

In [62]:
class Adaline:
  def __init__(self, learning_rate=0.001, n_iter = 1000, random_seed = 28):
    self.learning_rate = learning_rate
    self.n_iter = n_iter
    self.random_seed = random_seed
    self.weights = None
    self.umbral = 0
    self.costes = []

  def fit(self, X, y):
    """ Método que entrena el modelo """
    # Inicializamos el modelo
    self.__initialize_Adaline(X)
    # Realizamos la modificación de los pesos n_iter veces
    for i in range(self.n_iter):
      # Obtenemos la salida
      salida = np.dot(X, self.weights) + self.umbral
      # Obtenemos los errrores
      errores = y - salida
      # Actualizamos el vector de pesos
      self.weights += self.learning_rate * X.T.dot(errores)
      # Actualizamos el umbral
      self.umbral += self.learning_rate * np.sum(errores)
      # Obtenemos y guardamos el coste para cada época
      coste = (errores**2).sum() / 2.0
      self.costes.append(coste)


  def __initialize_Adaline(self, X):
    """ Método privado que inicializa los pesos y el umbral del Adaline de forma aleatoria """
    # Inicializamos la semilla de aleatoriedad
    np.random.seed(self.random_seed)
    # Generamos tantos números aleatorios desde -1 hasta 1 como atributos tenga el conjunto
    self.weights = np.random.uniform(-1, 1, size=(X.shape[1]))
    # Generamos el umbral de manera aleatoria
    self.umbral = np.random.uniform(-1, 1)

Comprobamos nuestra implementación del Adaline utilizando los datos artificiales proporcionados en aulaglobal.

In [63]:
data = pd.read_csv("datosArtificiales.txt", delimiter=' ', header=None)

X = data.iloc[:,:2]
print(X.head(5))

y = data.iloc[:,-1]
print(y.head(6))

          0         1
0  0.075144  0.208514
1  0.955954  0.504891
2  0.126028  0.301368
3  0.584149  0.734279
4  0.298572  0.491662
0    0.941773
1    2.407016
2    0.950687
3    1.434018
4    1.105482
5    0.758963
Name: 2, dtype: float64


In [64]:
modelo = Adaline()
modelo.fit(X,y)
print(modelo.costes)

[2076.3280166311943, 634.0853402876381, 245.86732324393924, 132.69625664990016, 92.69588647498983, 73.3232409296947, 60.71413214901114, 51.053217610814826, 43.16149666885429, 36.57297616953956, 31.03212482700233, 26.359649285452264, 22.414230841168155, 19.07966197011835, 16.259036934719198, 13.87117197598524, 11.847940927613841, 10.132130919688354, 8.675669584028386, 7.438146588340415, 6.385577145660816, 5.489366871252716, 4.725444827607409, 4.073537238028926, 3.516558908036073, 3.040103164688498, 2.6320142661912365, 2.2820288586713238, 1.9814752508108004, 1.7230211112575642, 1.5004617275390073, 1.3085422478864155, 1.1428084001319776, 0.9994810790878005, 0.8753509443280545, 0.7676897981199144, 0.6741760384881308, 0.5928319218558211, 0.5219707374274238, 0.46015230321198386, 0.40614545114344286, 0.3588963843472265, 0.3175019700916176, 0.28118718308595203, 0.2492860403433802, 0.2212254748256718, 0.19651168388634033, 0.17471856293201493, 0.15547789707717863, 0.13847103583695977, 0.12342181

Podemos observar que, efectivamente, el error cuadrático medio desciende en cada época, y se va aproximando a 0

Vamos a dividir los datos en conjuntos train, test y validacion


In [65]:
train_ratio = 0.7
test_ratio = 0.15
validation_ratio = 0.15

train_df, test_validation_df = train_test_split(datos, test_size=(test_ratio + validation_ratio), random_state=123)
test_df, validation_df = train_test_split(test_validation_df, test_size=(validation_ratio / (test_ratio + validation_ratio)), random_state=123)
